In [1]:
import pandas as pd
import numpy as np

In [2]:
df10 = pd.read_csv('./01_raw_data/EV_project/기후·환경/미세먼지10.csv', encoding='cp949')
df25 = pd.read_csv('./01_raw_data/EV_project/기후·환경/미세먼지2.5.csv', encoding='cp949')

In [3]:
df25 = df25.rename(columns={"구분(1)": "지역"})
df10 = df10.rename(columns={"구분(1)": "지역"})

df25_long = df25.melt(
    id_vars=["지역"],
    var_name="년월",
    value_name="미세먼지2.5 수치"
)
df10_long = df10.melt(
    id_vars=["지역"],
    var_name="년월",
    value_name="미세먼지10 수치"
)
df25_long[["년", "월"]] = (
    df25_long["년월"]
    .str.split(r"\.", expand=True)
    .astype(int)
)
df10_long[["년", "월"]] = (
    df10_long["년월"]
    .str.split(r"\.", expand=True)
    .astype(int)
)
df25_long = df25_long.drop(columns="년월")
df10_long = df10_long.drop(columns="년월")

df = pd.merge(
    df25_long,
    df10_long,
    on=["지역", "년", "월"],
    how="inner"
)

df["미세먼지2.5 수치"] = (
    df["미세먼지2.5 수치"]
    .astype(str)    
    .str.replace("*", "", regex=False) 
    .replace("", pd.NA) 
    .astype(float) 
)
df["미세먼지10 수치"] = (
    df["미세먼지10 수치"]
    .astype(str)  
    .str.replace("*", "", regex=False) 
    .replace("", pd.NA) 
    .astype(float) 
)

df = df[["지역", "년", "월", "미세먼지2.5 수치", "미세먼지10 수치"]]
df = df.replace('총계', '전국')

abbr_map = {
    '서울특별시' : '서울',
    '부산광역시' : '부산',
    '대구광역시' : '대구',
    '인천광역시' : '인천',
    '광주광역시' : '광주',
    '대전광역시' : '대전',
    '울산광역시' : '울산',
    '세종특별자치시' : '세종',
    '경기도' : '경기',
    '강원특별자치도' : '강원', 
    '충청북도' : '충북',
    '충청남도' : '충남',
    '전북특별자치도' :'전북',
    '전라남도' :'전남',
    '경상북도' : '경북',
    '경상남도' : '경남',
    '제주특별자치도': '제주'
}

df['지역'] = df['지역'].replace(abbr_map)

print(df['지역'].unique())

df


['전국' '서울' '부산' '대구' '인천' '광주' '대전' '울산' '세종' '경기' '강원' '충북' '충남' '전북'
 '전남' '경북' '경남' '제주']


,지역,년,월,미세먼지2.5 수치,미세먼지10 수치
0,전국,2018,11,32.0,56.0
1,서울,2018,11,28.0,52.0
2,부산,2018,11,26.0,47.0
3,대구,2018,11,31.0,57.0
4,인천,2018,11,29.0,55.0
...,...,...,...,...,...
1327,전북,2024,12,16.0,29.0
1328,전남,2024,12,12.0,22.0
1329,경북,2024,12,17.0,30.0
1330,경남,2024,12,13.0,21.0


In [4]:
df.to_csv('./02_interim_data/dust_data.csv', index=False, encoding='utf-8-sig')